In [13]:
import psycopg2
import sys
import pprint
from datetime import date, timedelta
import pandas as pd

import matplotlib.pyplot as plt
import os,sys
import matplotlib.patches as mpatches

#sys.path.append("../Mfig/")
#import mplp
from itertools import chain
import numpy as np
import csv
from collections import defaultdict
from datetime import date, timedelta
from datetime import datetime
from scipy.stats import entropy
%matplotlib inline

## DB connection

In [14]:
def conectionDB():
    conn_string = "dbname='stic' port='5432' user='stic' password='stic2019' host='gpmaster.as-dell.copernic.local'"
    #print ("Connecting to database ->{}".format(conn_string))
    conn = psycopg2.connect(conn_string)
    return conn

# build dictionaries


In [15]:
dict_dpto = dict()
dict_province = dict()
dict_district = dict()
with open('../data/ubigeo_inei.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        #print (row)
        if (str(row[0])!=''):
            dict_dpto[str(row[3]).lower()]=(str(row[0]))
        if (str(row[1])!=''):
            dict_province[(row[4]).lower()]=(row[0]+''+row[1])
        if (str(row[2])!=''):
            dict_district[(row[5]).lower()]=(row[0]+''+row[1]+''+row[2])
        
dict_gender = {'M':1,'F':0,'':-1}
dict_card = {'TD':1,'TC':0,'':-1}
dict_region = {'NORTE':0,'ORIENTE':1,'SUR':2,'CENTRO':3,'SIERRA CENTRAL':4,
               'LIMA CENTRO':5,'LIMA ESTE':6,'LIMA SUR':7,'LIMA NORTE':8,
               'LIMA MODERNA':9,'LIMA PROVINCIA':10,'CALLAO':11,'':12}


In [16]:
columnNames=[ 
        'social_class' ,     #-- 0
        'country_code',      #-- 1
        'amount_usd',        #-- 2 *
        'client_age',        #-- 3 *
        'client_gender',     #-- 4
        'debit_type',        #-- 5 Credit or debit card
        'agency_departement',#-- 6 
        'agency_province',   #-- 7
        'agency_district',   #-- 8
        'agency_region',     #-- 9
        'merchant_departement', #-- 10
        'merchant_province', #-- 11
        'merchant_district' #-- 12
        ]


## extraction loop

In [17]:
def extract_all (index):
    PLIMIT = 1000000
    POFFSET = 1000000
    #NUM = 93 # Millions of records (92 470 071)
    NUM = 2 #for test 
    i=index
    while (i < NUM):
        print ("Processing {}/{} ...".format(i,NUM))
        query ="""
            SELECT 
                social_class,     -- 0
                country_code,     -- 1
                amount_usd,        -- 2 *
                client_age,        -- 3 *
                client_gender,     -- 4
                debit_type,        -- 5 Credit or debit card
                agency_departement,-- 6 
                agency_province,   -- 7
                agency_district,   -- 8
                agency_region,     -- 9
                merchant_departement, -- 10
                merchant_province, -- 11
                merchant_district -- 12
            FROM 
                public.bbva 
            INNER JOIN 
                client
            ON 
                client.client_id = bbva.client_id
            WHERE
                bbva.merchant_id != '00000000'
            ORDER BY 
                bbva.date ASC
            LIMIT {}
            OFFSET {}
        ;
        """.format(PLIMIT,POFFSET*i)
        conn = conectionDB()
        cursor = conn.cursor();
        df = pd.read_sql(query, conn)
        df.fillna(-1, inplace=True)
        
        df['small_social_class'] = df.apply(lambda row: 1 if (row.social_class<=3) else (3 if (row.social_class>=7) else (1 if (row.social_class<=0) else 2) ), axis=1)
        del df['social_class']
        df['country_code'] = df.apply(lambda row: 1 if (row.country_code=='PE') else 0 , axis=1)
      
        df["agency_departement"] = df["agency_departement"].str.lower()
        df["agency_province"] = df["agency_province"].str.lower()
        df["agency_district"] = df["agency_district"].str.lower()
        df["merchant_departement"] = df["merchant_departement"].str.lower()
        df["merchant_province"] = df["merchant_province"].str.lower()
        df["merchant_district"] = df["merchant_district"].str.lower()

        df=df.replace({"agency_region":dict_region,
                       "client_gender": dict_gender,
                          "debit_type":dict_card,
                         "agency_departement":dict_dpto,
                         "agency_province":dict_province,
                         "agency_district":dict_district,
                          "merchant_departement":dict_dpto,
                         "merchant_province":dict_province,
                         "merchant_district":dict_district
                         })
        
        df.fillna(-1, inplace=True)

        df.to_csv (r'../data_output/export_dataframe_{}v3.csv'.format(i), index = None, header=False)
        i+=1

In [18]:
extract_all(0)

Processing 0/2 ...
Processing 1/2 ...
